# Data Preprocessing Pipeline

**COMP647 Assignment 02 - Student ID: 1163127**

This notebook implements comprehensive data preprocessing for Lending Club loan data analysis.

## 1. Import Libraries and Setup

In [ ]:
# Essential data processing libraries
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning utilities
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy import stats

# System and utility libraries
import warnings
import os
from pathlib import Path

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('default')

## 2. Data Loading Functions

In [ ]:
def load_sample_data(sample_size='10000'):
    """
    Load sample datasets for development and analysis.
    
    Parameters:
    sample_size (str): Size of sample to load ('1000', '10000', '50000')
    
    Returns:
    tuple: (accepted_df, rejected_df)
    """
    # Define file paths for data loading
    data_path = '../data/processed/'
    accepted_file = f'accepted_sample_{sample_size}.csv'
    rejected_file = f'rejected_sample_{sample_size}.csv'
    
    print(f"Loading sample datasets (size: {sample_size})...")
    
    try:
        # Load accepted loans dataset
        accepted_df = pd.read_csv(os.path.join(data_path, accepted_file))
        print(f"Accepted loans loaded: {accepted_df.shape[0]:,} rows, {accepted_df.shape[1]} columns")
        
        # Load rejected loans dataset
        rejected_df = pd.read_csv(os.path.join(data_path, rejected_file))
        print(f"Rejected loans loaded: {rejected_df.shape[0]:,} rows, {rejected_df.shape[1]} columns")
        
        print("Data loading completed successfully!")
        
        return accepted_df, rejected_df
        
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        print("Please ensure data files are in the correct directory")
        return None, None
    except Exception as e:
        print(f"Unexpected error during data loading: {e}")
        return None, None

## 3. Missing Value Analysis

In [ ]:
def analyze_missing_values(df):
    """
    Comprehensive analysis of missing values in the dataset.
    
    Parameters:
    df (pd.DataFrame): Input dataframe
    
    Returns:
    dict: Missing value analysis results
    """
    print(f"Analyzing missing values for dataset with shape: {df.shape}")
    
    # Calculate missing values for each column
    missing_data = df.isnull().sum()
    missing_percentage = (missing_data / len(df)) * 100
    
    # Create detailed missing data summary
    missing_summary = pd.DataFrame({
        'Column': missing_data.index,
        'Missing_Count': missing_data.values,
        'Missing_Percentage': missing_percentage.values,
        'Data_Type': df.dtypes.values,
        'Unique_Values': [df[col].nunique() for col in df.columns]
    })
    
    # Filter only columns with missing values
    missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
    missing_summary = missing_summary.sort_values('Missing_Percentage', ascending=False)
    
    # Calculate summary statistics
    total_missing = missing_data.sum()
    columns_with_missing = len(missing_summary)
    overall_missing_pct = (total_missing / (len(df) * len(df.columns))) * 100
    
    print(f"Columns with missing values: {columns_with_missing}")
    print(f"Total missing values: {total_missing:,}")
    print(f"Overall missing percentage: {overall_missing_pct:.2f}%")
    
    # Categorize missing values by severity
    analysis_results = {
        'missing_summary': missing_summary,
        'total_missing_values': total_missing,
        'columns_with_missing': columns_with_missing,
        'overall_missing_percentage': overall_missing_pct,
        'structural_missing_90plus': len(missing_summary[missing_summary['Missing_Percentage'] > 90]),
        'high_missing_50_90': len(missing_summary[(missing_summary['Missing_Percentage'] > 50) & (missing_summary['Missing_Percentage'] <= 90)]),
        'moderate_missing_10_50': len(missing_summary[(missing_summary['Missing_Percentage'] > 10) & (missing_summary['Missing_Percentage'] <= 50)]),
        'low_missing_under_10': len(missing_summary[missing_summary['Missing_Percentage'] <= 10])
    }
    
    if columns_with_missing > 0:
        print("\nMissing value categorization:")
        print(f"  Structural missing (>90%): {analysis_results['structural_missing_90plus']} columns")
        print(f"  High missing (50-90%): {analysis_results['high_missing_50_90']} columns") 
        print(f"  Moderate missing (10-50%): {analysis_results['moderate_missing_10_50']} columns")
        print(f"  Low missing (<10%): {analysis_results['low_missing_under_10']} columns")
        
        print(f"\nTop 10 columns with highest missing percentages:")
        if len(missing_summary) > 0:
            for _, row in missing_summary.head(10).iterrows():
                print(f"  {row['Column']}: {row['Missing_Percentage']:.1f}% ({row['Missing_Count']:,} values)")
    else:
        print("No missing values found in dataset")
    
    return analysis_results

## 4. Data Preprocessing Pipeline

In [ ]:
def preprocess_lending_data(df):
    """
    Main preprocessing pipeline for Lending Club data.
    
    Parameters:
    df (pd.DataFrame): Raw lending data
    
    Returns:
    pd.DataFrame: Preprocessed data
    """
    print(f"Starting preprocessing pipeline for dataset: {df.shape}")
    
    # Step 1: Create a copy to avoid modifying original data
    df_processed = df.copy()
    
    # Step 2: Basic data validation
    print("Step 1: Basic data validation")
    print(f"  Original shape: {df_processed.shape}")
    print(f"  Memory usage: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Step 3: Remove duplicate rows
    print("Step 2: Duplicate removal")
    initial_rows = len(df_processed)
    df_processed = df_processed.drop_duplicates()
    duplicates_removed = initial_rows - len(df_processed)
    print(f"  Duplicates removed: {duplicates_removed:,}")
    
    # Step 4: Analyze missing values
    print("Step 3: Missing value analysis")
    missing_analysis = analyze_missing_values(df_processed)
    
    # Step 5: Handle missing values based on analysis
    print("Step 4: Missing value treatment")
    if missing_analysis['columns_with_missing'] > 0:
        print("  Missing value treatment will be implemented in subsequent steps")
        # Placeholder for missing value treatment implementation
    else:
        print("  No missing values found - skipping treatment")
    
    # Step 6: Data type optimization preparation
    print("Step 5: Data type optimization")
    print(f"  Current data types: {df_processed.dtypes.value_counts().to_dict()}")
    
    print(f"Preprocessing pipeline completed. Final shape: {df_processed.shape}")
    
    return df_processed

## 5. Outlier Detection Functions

Outlier detection using statistical methods for data quality assessment.

In [ ]:
def detect_outliers_iqr(df, columns=None, multiplier=1.5):
    """
    Detect outliers using Interquartile Range (IQR) method
    
    This function identifies outliers by calculating the IQR for numeric columns
    and flagging values that fall outside the bounds defined by Q1 - multiplier*IQR
    and Q3 + multiplier*IQR.
    
    Parameters:
    df (DataFrame): Input dataset for outlier analysis
    columns (list): List of column names to analyze (None for all numeric columns)  
    multiplier (float): IQR multiplier for outlier threshold (default: 1.5)
    
    Returns:
    dict: Dictionary containing outlier information for each analyzed column
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    outlier_info = {}
    
    for col in columns:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            
            outlier_info[col] = {
                'count': len(outliers),
                'percentage': (len(outliers) / len(df)) * 100,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'Q1': Q1,
                'Q3': Q3,
                'IQR': IQR
            }
    
    return outlier_info

def detect_outliers_zscore(df, columns=None, threshold=3):
    """
    Detect outliers using Z-score method
    
    This function identifies outliers using standardized Z-scores, flagging values
    that have an absolute Z-score greater than the specified threshold.
    
    Parameters:
    df (DataFrame): Input dataset for outlier analysis
    columns (list): List of column names to analyze (None for all numeric columns)
    threshold (float): Z-score threshold for outlier detection (default: 3)
    
    Returns:
    dict: Dictionary containing outlier information for each analyzed column
    """
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    outlier_info = {}
    
    for col in columns:
        if col in df.columns:
            z_scores = np.abs(stats.zscore(df[col].dropna()))
            outliers_mask = z_scores > threshold
            outlier_count = outliers_mask.sum()
            
            outlier_info[col] = {
                'count': outlier_count,
                'percentage': (outlier_count / len(df[col].dropna())) * 100,
                'threshold': threshold,
                'max_zscore': z_scores.max() if len(z_scores) > 0 else 0
            }
    
    return outlier_info

print("Outlier detection functions defined successfully")
print("Available methods: detect_outliers_iqr(), detect_outliers_zscore()")

## 6. Main Execution

This section demonstrates the complete preprocessing workflow using the implemented functions.

In [ ]:
# Load sample data for demonstration
print("=== LOADING SAMPLE DATA ===")
df_accepted, df_rejected = load_sample_data(sample_size='10000')

if df_accepted is not None and df_rejected is not None:
    print(f"\nAccepted loans dataset shape: {df_accepted.shape}")
    print(f"Rejected loans dataset shape: {df_rejected.shape}")
    
    # Display basic info about accepted dataset
    print("\n=== ACCEPTED DATASET OVERVIEW ===")
    print(f"Data types: {df_accepted.dtypes.value_counts().to_dict()}")
    print(f"Memory usage: {df_accepted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\nFirst few columns:")
    print(df_accepted.columns[:10].tolist())
else:
    print("Failed to load data - check file paths and ensure sample files exist")

In [ ]:
# Analyze missing values in the accepted dataset
if df_accepted is not None:
    print("=== MISSING VALUE ANALYSIS ===")
    missing_results = analyze_missing_values(df_accepted)
    
    # Display detailed results
    print(f"\nAnalysis Summary:")
    print(f"- Total columns with missing data: {missing_results['columns_with_missing']}")
    print(f"- Overall missing percentage: {missing_results['overall_missing_percentage']:.2f}%")
    
    if missing_results['columns_with_missing'] > 0:
        print(f"\nMissing data severity breakdown:")
        print(f"- Structural missing (>90%): {missing_results['structural_missing_90plus']} columns")
        print(f"- High missing (50-90%): {missing_results['high_missing_50_90']} columns")
        print(f"- Moderate missing (10-50%): {missing_results['moderate_missing_10_50']} columns") 
        print(f"- Low missing (<10%): {missing_results['low_missing_under_10']} columns")
        
        # Show the missing summary dataframe if it exists
        if len(missing_results['missing_summary']) > 0:
            print(f"\nTop 5 columns with highest missing percentages:")
            top_missing = missing_results['missing_summary'].head(5)
            for _, row in top_missing.iterrows():
                print(f"- {row['Column']}: {row['Missing_Percentage']:.1f}% missing ({row['Data_Type']})")
else:
    print("No data available for missing value analysis")

In [ ]:
# Run the complete preprocessing pipeline
if df_accepted is not None:
    print("=== PREPROCESSING PIPELINE DEMONSTRATION ===")
    df_processed = preprocess_lending_data(df_accepted)
    
    print(f"\n=== PREPROCESSING RESULTS ===")
    print(f"Original shape: {df_accepted.shape}")
    print(f"Processed shape: {df_processed.shape}")
    
    # Compare memory usage
    original_memory = df_accepted.memory_usage(deep=True).sum() / 1024**2
    processed_memory = df_processed.memory_usage(deep=True).sum() / 1024**2
    print(f"Original memory usage: {original_memory:.2f} MB")
    print(f"Processed memory usage: {processed_memory:.2f} MB")
    
    if df_processed.shape[0] < df_accepted.shape[0]:
        rows_removed = df_accepted.shape[0] - df_processed.shape[0]
        print(f"Rows removed during processing: {rows_removed:,}")
    
    print("\nPreprocessing pipeline demonstration completed!")
else:
    print("No data available for preprocessing pipeline demonstration")